# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

## Basic FABRIC Slice Configuration

In [1]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MASS         4  125/128  1016/1024  55750/55800    253/254                 2/2                             0/0                            6/6                 0/0               3/3
MAX         10  248/320  2326/2560  115058/116400  579/635                 2/2                             4/4                            16/16               4/4               6/6
TACC        10  306/320  2516/2560  116350/116400  633/635                 0/2                             1/4                            16/16               4/4               6/6
MICH         6  192/192  1536/1536  60600/60600    381/381                 2/2

## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [3]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv6Network, IPv4Network

# Slice 
slice_name = 'VCCchernovik'

# [site1,site2,site3] = fablib.get_random_sites(count=3)
site1 = site2 = site3 = 'MAX'
print(f"Sites: {site1},{site2},{site3}")


# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 100

# Hosts
h1_name = "h1"
h11_name = "h11"
h2_name = "h2"
h22_name = "h22"


# h1_subnet=IPv4Network('10.0.0.0/16')
# h1_addr=IPv4Address('10.0.1.1')
# h11_addr=IPv4Address('10.0.1.11')

# h2_subnet=h1_subnet #IPv4Network('10.0.0.0/16')
# h2_addr=IPv4Address('10.0.2.2')
# h22_addr=IPv4Address('10.0.2.22')

host_cores = 2
host_ram = 8
host_disk = 20

net_h1_name = 'net_h1'
net_h11_name = 'net_h11'
net_h2_name = 'net_h2'
net_h22_name = 'net_h22'

net_s1_s2_name = 'net_s1_s2'
net_s2_s3_name = 'net_s2_s3'
net_s1_s3_name = 'net_s1_s3'

s1_iface_h1,s1_iface_h11,s1_iface_to_s2,s1_iface_to_s3 = "s1_iface_h1","s1_iface_h11","s1_iface_to_s2","s1_iface_to_s3"  
s2_iface_h2,s2_iface_h22,s2_iface_to_s1,s2_iface_to_s3 = "s2_iface_h2","s2_iface_h22","s2_iface_to_s1","s2_iface_to_s3" 
s3_iface_to_s1,s3_iface_to_s2 = "s3_iface_to_s1","s3_iface_to_s2"

# All node properties
image = 'default_ubuntu_20'

Sites: MAX,MAX,MAX


### Create the Slice

In [4]:
import datetime
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    def addSwitchNIC(nodeName,site,ifaceNames):
        node = slice.add_node(name=nodeName, site=site,  image=image, cores=switch_cores, ram=switch_ram, disk=switch_disk)
        ifaceObj = {}
        for name in ifaceNames:
            iface =  node.add_component(model='NIC_Basic', name=name).get_interfaces()[0]
            ifaceObj[name] = iface
        return node, ifaceObj
    
    # Add switch node s1
    s1, s1_ifaces = addSwitchNIC(s1_name,site1,[s1_iface_h1,s1_iface_h11,s1_iface_to_s2,s1_iface_to_s3])

    # Add switch node s2
    s2, s2_ifaces = addSwitchNIC(s2_name,site2,[s2_iface_h2,s2_iface_h22,s2_iface_to_s1,s2_iface_to_s3])
    
    # Add switch node s3
    s3, s3_ifaces = addSwitchNIC(s3_name,site3,[s3_iface_to_s1,s3_iface_to_s2])
    
    def addHostNIC(nodeName,componentName,site):
        node = slice.add_node(name=nodeName, site=site, image=image, cores=host_cores, ram=host_ram, disk=host_disk)
        iface = node.add_component(model='NIC_Basic', name=componentName).get_interfaces()[0]
        return node,iface
    
    # Add host node h1
    h1,h1_iface = addHostNIC(h1_name,"h1_nic",site1)

    # Add host node h11
    h11 , h11_iface = addHostNIC(h11_name,"h11_nic",site1)
    
    # Add host node h2
    h2, h2_iface = addHostNIC(h2_name,"h2_nic",site2)
    
    # Add host node h3
    h22, h22_iface = addHostNIC(h22_name,"h22_nic",site2)
    
    #Add swtich networks
    switch_net1 = slice.add_l3network(name=net_s1_s2_name, interfaces=[s1_ifaces[s1_iface_to_s2], s2_ifaces[s2_iface_to_s1]], type='IPv4')
    swtich_net2 = slice.add_l3network(name=net_s2_s3_name, interfaces=[s2_ifaces[s2_iface_to_s3], s3_ifaces[s3_iface_to_s2]], type='IPv4')
    swtich_net3 = slice.add_l3network(name=net_s1_s3_name, interfaces=[s3_ifaces[s3_iface_to_s1], s1_ifaces[s1_iface_to_s3]], type='IPv4')

    #Add host networks 
    host_net1 = slice.add_l3network(name=net_h1_name, interfaces=[s1_ifaces[s1_iface_h1], h1_iface], type='IPv4')
    host_net11 = slice.add_l3network(name=net_h11_name, interfaces=[s1_ifaces[s1_iface_h11], h11_iface], type='IPv4')
    host_net2 = slice.add_l3network(name=net_h2_name, interfaces=[s2_ifaces[s2_iface_h2], h2_iface], type='IPv4')
    host_net22 = slice.add_l3network(name=net_h22_name, interfaces=[s2_ifaces[s2_iface_h22], h22_iface], type='IPv4')
    
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


-----------  ------------------------------------
Slice Name   VCCchernovik
Slice ID     edbe999e-6ea1-4ef8-b924-0cb195253cf1
Slice State  StableOK
Lease End    2022-09-05 17:44:00 +0000
-----------  ------------------------------------

Retry: 19, Time: 268 sec

ID                                    Name    Site    Host                         Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  ------  ------  -------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
748a23e9-f958-488b-81c9-a0e52c761c57  s1      MAX     max-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  63.239.135.122   Active
de2ebe97-2e65-4608-9de2-fa9dfe84aa81  s2      MAX     max-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  63.239.135.120   Active
3800e7ba-0c20-4855-a0bc-6e58a68026d8  s3      MAX     max-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  6

In [5]:
# #Get swtich networks
# switch_net1 = slice.get_network(name=net_s1_s2_name)
# switch_net1_available_ips = switch_net1.get_available_ips()
# swtich_net2 = slice.get_network(name=net_s2_s3_name)
# swtich_net2_available_ips = swtich_net2.get_available_ips()
# swtich_net3 = slice.get_network(name=net_s1_s3_name)
# swtich_net3_available_ips = swtich_net3.get_available_ips()

# #Get host networks 
# host_net1 = slice.get_network(name=net_h1_name)
# host_net1_available_ips = host_net1.get_available_ips()
# host_net11 = slice.get_network(name=net_h11_name)
# host_net11_available_ips = host_net11.get_available_ips()
# host_net2 = slice.get_network(name=net_h2_name)
# host_net2_available_ips = host_net2.get_available_ips()
# host_net22 = slice.get_network(name=net_h22_name)
# host_net22_available_ips = host_net22.get_available_ips()

In [6]:
# print(slice.get_network(name=net_h1_name))


In [7]:
config_threads = {}

In [8]:
def checkInetConnection(node):
    try:
        #If the node is an IPv4 Node then configure NAT64
        if type(ip_address(node.get_management_ip())) is IPv6Address:
            print("This code only works with ip4, reconfigure nodes")
            return

        #Access non-IPv4 Services
        config_thread = node.execute_thread(f'sudo apt-get update -qq && sudo apt install net-tools && git clone https://github.com/fabric-testbed/jupyter-examples.git && ls jupyter-examples')
        config_threads[node]=config_thread

    except Exception as e:
        print(f"Exception: {e}")

In [9]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        checkInetConnection(node)
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [10]:
def printThreadOutputs(con_threads):
    try: 
        for node, thread in con_threads.items():
            stdout, stderr = thread.result()
            print(f"Config thread node {node.get_name()} complete")
            # checkIP(node)
    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc() 
printThreadOutputs(config_threads)
config_threads = {}    

Config thread node s1 complete
Config thread node s2 complete
Config thread node s3 complete
Config thread node h1 complete
Config thread node h11 complete
Config thread node h2 complete
Config thread node h22 complete


### Print Details

In [11]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("----")
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        # print(f"   Cores             : {node.get_cores()}")
        # print(f"   RAM               : {node.get_ram()}")
        # print(f"   Disk              : {node.get_disk()}")
        # print(f"   Image             : {node.get_image()}")
        # print(f"   Image Type        : {node.get_image_type()}")
        # print(f"   Host              : {node.get_host()}")
        # print(f"   Site              : {node.get_site()}")
        # print(f"   Management IP     : {node.get_management_ip()}")
        # print(f"   Reservation ID    : {node.get_reservation_id()}")
        # print(f"   Reservation State : {node.get_reservation_state()}")
        ssh = node.get_ssh_command().replace("ssh -i /home/fabric/work/fabric_config/.ssh/id_rsa -J durbek_gafurov_0000000854@bastion-1.fabric-testbed.net","./connect.sh")
        print(f"   SSH Command       : {ssh}")
        # print(f"   Components        :  ")
        # for component in node.get_components():
        #     print(f"      Name             : {component.get_name()}")
        #     print(f"      Details          : {component.get_details()}")
        #     print(f"      Disk (G)         : {component.get_disk()}")
        #     print(f"      Units            : {component.get_unit()}")
        #     print(f"      PCI Address      : {component.get_pci_addr()}")
        #     print(f"      Model            : {component.get_model()}")
        #     print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ", end = " ")
        for interface in node.get_interfaces():
        #     print(f"       Name                : {interface.get_name()}")
        #     print(f"           Bandwidth           : {interface.get_bandwidth()}")
            # print(f"           VLAN                : {interface.get_vlan()}")  
            # print(f"           MAC                 : {interface.get_mac()}") 
            print(interface.get_os_interface(), end =" ")
    # for network in slice.get_l2networks():
    #     print("Network:")
    #     print(f"    Name:            {network.get_name()}")
    # print(f"Interface Map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

----
Node:
   Name              : s1
   SSH Command       : ./connect.sh ubuntu@63.239.135.122
   Interfaces        :   ens7 ens9 ens8 ens10 ----
Node:
   Name              : s2
   SSH Command       : ./connect.sh ubuntu@63.239.135.120
   Interfaces        :   ens7 ens10 ens8 ens9 ----
Node:
   Name              : s3
   SSH Command       : ./connect.sh ubuntu@63.239.135.87
   Interfaces        :   ens7 ens8 ----
Node:
   Name              : h1
   SSH Command       : ./connect.sh ubuntu@63.239.135.82
   Interfaces        :   ens7 ----
Node:
   Name              : h11
   SSH Command       : ./connect.sh ubuntu@63.239.135.101
   Interfaces        :   ens7 ----
Node:
   Name              : h2
   SSH Command       : ./connect.sh ubuntu@63.239.135.70
   Interfaces        :   ens7 ----
Node:
   Name              : h22
   SSH Command       : ./connect.sh ubuntu@63.239.135.93
   Interfaces        :   ens7 

## Configure Nodes


In [12]:
config_threads = {}

In [13]:
"""
try:
    node1_iface = node1.get_interface(network_name=network1_name)  
    node1_addr = network1_available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=network1.get_subnet())
    
    node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node1.execute(f'ip route list')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")
"""

'\ntry:\n    node1_iface = node1.get_interface(network_name=network1_name)  \n    node1_addr = network1_available_ips.pop(0)\n    node1_iface.ip_addr_add(addr=node1_addr, subnet=network1.get_subnet())\n    \n    node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())\n    \n    stdout, stderr = node1.execute(f\'ip addr show {node1_iface.get_os_interface()}\')\n    print (stdout)\n    \n    stdout, stderr = node1.execute(f\'ip route list\')\n    print (stdout)\nexcept Exception as e:\n    print(f"Exception: {e}")\n'

In [14]:
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/ && ls tutorials" 
def addIP2(name1,name2,net_name):
    print(name1,"-",name2)
    network = slice.get_network(name=net_name)
    network_available_ips = network.get_available_ips() 
    # Node 1
    node1 = slice.get_node(name=name1) 
    node1_iface = node1.get_interface(network_name=net_name)    
    node1_iface.ip_addr_add(addr=network_available_ips.pop(0), subnet=network.get_subnet())
    node1.ip_route_add(subnet=network.get_subnet(), gateway=network.get_gateway())
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    ip1 = stdout.split("inet ")[1].split(" scope ")[0]
    print (ip1)
    stdout, stderr = node1.execute(f'ip route list | grep {node1_iface.get_os_interface()}')
    print (stdout)
    config_threads[node1]=node1.execute_thread(host_config_script)
    # Node 2 
    node2 = slice.get_node(name=name2)  
    node2_iface = node2.get_interface(network_name=net_name)  
    node2_iface.ip_addr_add(addr=network_available_ips.pop(0), subnet=network.get_subnet())
    node2.ip_route_add(subnet=network.get_subnet(), gateway=network.get_gateway())
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    ip2 = stdout.split("inet ")[1].split(" scope ")[0]
    print (ip2)
    stdout, stderr = node2.execute(f'ip route list | grep {node2_iface.get_os_interface()}')
    print (stdout)    
    return node1, node2, ip1.split("/")[0]+"/32", ip2
    
try:
    
    h1,s1, h1IP, s1h1IP = addIP2(h1_name,s1_name,net_h1_name)
    h11,s1, h11IP, s1h11IP = addIP2(h11_name,s1_name,net_h11_name)
    h2,s2, h2IP, s2h2IP = addIP2(h2_name,s2_name,net_h2_name)
    h22,s2, h22IP,s2h22IP = addIP2(h22_name,s2_name,net_h22_name)
    
    s1, s2, s1s2IP,s2s1IP = addIP2(s1_name,s2_name,net_s1_s2_name)
    s2, s3, s2s3IP,s3s2IP = addIP2(s2_name,s3_name,net_s2_s3_name)
    s3, s1,s3s1IP,s1s3IP = addIP2(s3_name,s1_name,net_s1_s3_name)


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

h1 - s1
10.130.4.2/24
10.130.4.0/24 dev ens7 proto kernel scope link src 10.130.4.2 

10.130.4.3/24
10.130.4.0/24 dev ens9 proto kernel scope link src 10.130.4.3 

h11 - s1
10.130.5.2/24
10.130.5.0/24 dev ens7 proto kernel scope link src 10.130.5.2 

10.130.5.3/24
10.130.5.0/24 dev ens10 proto kernel scope link src 10.130.5.3 

h2 - s2
10.130.6.2/24
10.130.6.0/24 dev ens7 proto kernel scope link src 10.130.6.2 

10.130.6.3/24
10.130.6.0/24 dev ens7 proto kernel scope link src 10.130.6.3 

h22 - s2
10.130.7.2/24
10.130.7.0/24 dev ens7 proto kernel scope link src 10.130.7.2 

10.130.7.3/24
10.130.7.0/24 dev ens8 proto kernel scope link src 10.130.7.3 

s1 - s2
10.130.1.2/24
10.130.1.0/24 dev ens7 proto kernel scope link src 10.130.1.2 

10.130.1.3/24
10.130.1.0/24 dev ens9 proto kernel scope link src 10.130.1.3 

s2 - s3
10.130.2.2/24
10.130.2.0/24 dev ens10 proto kernel scope link src 10.130.2.2 

10.130.2.3/24
10.130.2.0/24 dev ens7 proto kernel scope link src 10.130.2.3 

s3 - s1
10.1

In [15]:
printThreadOutputs(config_threads)
config_threads = {}

Config thread node h1 complete
Config thread node h11 complete
Config thread node h2 complete
Config thread node h22 complete
Config thread node s1 complete
Config thread node s2 complete
Config thread node s3 complete


## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



Below are commands to let sudo work with the global proxy.

In [16]:
print(slice.get_node(name=s1_name).get_interface(network_name=net_s1_s2_name))
print(h1.get_interface(network_name=net_h1_name))

---------------------  --------------------
Name                   s1-s1_iface_to_s2-p1
Network                net_s1_s2
Bandwidth              0
VLAN
MAC                    1E:98:AB:0A:2D:14
Physical OS Interface  ens7
OS Interface           ens7
---------------------  --------------------
---------------------  -----------------
Name                   h1-h1_nic-p1
Network                net_h1
Bandwidth              0
VLAN
MAC                    2A:5F:FB:92:07:F6
Physical OS Interface  ens7
OS Interface           ens7
---------------------  -----------------


In [17]:
h1Mac = h1.get_interface(network_name=net_h1_name).get_mac()
h11Mac = h11.get_interface(network_name=net_h11_name).get_mac()
h2Mac = h2.get_interface(network_name=net_h2_name).get_mac()
h22Mac = h22.get_interface(network_name=net_h22_name).get_mac()

s1_net_s1_s2_Mac = s1.get_interface(network_name=net_s1_s2_name).get_mac()
s2_net_s1_s2_Mac = s2.get_interface(network_name=net_s1_s2_name).get_mac()

s1_net_s1_s3_Mac = s1.get_interface(network_name=net_s1_s3_name).get_mac()
s3_net_s1_s3_Mac = s3.get_interface(network_name=net_s1_s3_name).get_mac()

s2_net_s2_s3_Mac = s2.get_interface(network_name=net_s2_s3_name).get_mac()
s3_net_s2_s3_Mac = s3.get_interface(network_name=net_s2_s3_name).get_mac()

print(h1Mac,h11Mac,h2Mac,h22Mac)
def printToFile(text,filename):
    text_file = open(filename, "w")
    n = text_file.write(text)
    text_file.close()
s1_command = """table_clear MyEgress.swtrace
table_clear MyIngress.ipv4_lpm 
table_set_default MyEgress.swtrace add_swtrace 1
table_add MyIngress.ipv4_lpm ipv4_forward {} => {} 1
table_add MyIngress.ipv4_lpm ipv4_forward {} => {} 2
table_add MyIngress.ipv4_lpm ipv4_forward {} => {} 3
table_add MyIngress.ipv4_lpm ipv4_forward {} => {} 4
""".format(h1IP,h1Mac,h11IP, h11Mac,h2IP,s2_net_s1_s2_Mac,h22IP,s3_net_s1_s3_Mac)
s2_command = """table_clear MyEgress.swtrace
table_clear MyIngress.ipv4_lpm 
table_set_default MyEgress.swtrace add_swtrace 2
table_add MyIngress.ipv4_lpm ipv4_forward {} => {} 3
table_add MyIngress.ipv4_lpm ipv4_forward {} => {} 4
table_add MyIngress.ipv4_lpm ipv4_forward {} => {} 1
table_add MyIngress.ipv4_lpm ipv4_forward {} => {} 2
""".format(h1IP,s1_net_s1_s2_Mac,h11IP,s3_net_s2_s3_Mac,h2IP,h2Mac,h22IP,h22Mac)
s3_command = """table_clear MyEgress.swtrace
table_clear MyIngress.ipv4_lpm 
table_set_default MyEgress.swtrace add_swtrace 3
table_add MyIngress.ipv4_lpm ipv4_forward {} => {} 1
table_add MyIngress.ipv4_lpm ipv4_forward {} => {} 2
""".format(h1IP,s1_net_s1_s3_Mac,h2IP,s2_net_s2_s3_Mac)
printToFile(s1_command, "./scripts/s1command.txt")
printToFile(s2_command, "./scripts/s2command.txt")
printToFile(s3_command, "./scripts/s3command.txt")
print(s1_command, "./scripts/s1command.txt")
print(s2_command, "./scripts/s2command.txt")
print(s3_command, "./scripts/s3command.txt")

2A:5F:FB:92:07:F6 2A:AE:C2:4C:53:D0 2E:FB:22:D0:52:69 32:38:2E:A2:3A:F5
table_clear MyEgress.swtrace
table_clear MyIngress.ipv4_lpm 
table_set_default MyEgress.swtrace add_swtrace 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.130.4.2/32 => 2A:5F:FB:92:07:F6 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.130.5.2/32 => 2A:AE:C2:4C:53:D0 2
table_add MyIngress.ipv4_lpm ipv4_forward 10.130.6.2/32 => 26:1E:1E:F8:9E:F8 3
table_add MyIngress.ipv4_lpm ipv4_forward 10.130.7.2/32 => 26:F7:5A:56:28:4C 4
 ./scripts/s1command.txt
table_clear MyEgress.swtrace
table_clear MyIngress.ipv4_lpm 
table_set_default MyEgress.swtrace add_swtrace 2
table_add MyIngress.ipv4_lpm ipv4_forward 10.130.4.2/32 => 1E:98:AB:0A:2D:14 3
table_add MyIngress.ipv4_lpm ipv4_forward 10.130.5.2/32 => 26:EE:CE:BD:92:82 4
table_add MyIngress.ipv4_lpm ipv4_forward 10.130.6.2/32 => 2E:FB:22:D0:52:69 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.130.7.2/32 => 32:38:2E:A2:3A:F5 2
 ./scripts/s2command.txt
table_clear MyEgress.swtra

In [18]:
def configureSwitch(name,network_names):
    try:
        node = slice.get_node(name=name)
        os_ifaces = ""
        for net_name in network_names:
            iface = node.get_interface(network_name=net_name).get_os_interface()
            if iface is None:
                print("iface is none")
                return
            os_ifaces=os_ifaces+iface+" "

        file_attributes = node.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
        command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {os_ifaces}  > /tmp/script.log 2>&1' && tail /tmp/script.log"
        config_threads[node] = node.execute_thread(command)
        print(command)


    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()

In [19]:
configureSwitch(s1_name,[net_h1_name,net_h11_name,net_s1_s2_name,net_s1_s3_name]) 
configureSwitch(s2_name,[net_h2_name,net_h22_name,net_s1_s2_name,net_s2_s3_name])
configureSwitch(s3_name,[net_s1_s3_name,net_s2_s3_name])

chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens9 ens10 ens7 ens8   > /tmp/script.log 2>&1' && tail /tmp/script.log
chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens7 ens8 ens9 ens10   > /tmp/script.log 2>&1' && tail /tmp/script.log
chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens8 ens7   > /tmp/script.log 2>&1' && tail /tmp/script.log


In [20]:
printThreadOutputs(config_threads)
config_threads = {}

Config thread node s1 complete
Config thread node s2 complete
Config thread node s3 complete


### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [21]:

for switch_name in [s1_name, s2_name, s3_name]:
    switch_node = slice.get_node(name=switch_name)
    management_ip_switch = str(switch_node.get_management_ip())
    print("Swtitch Name        : {}".format(switch_node.get_name()))
    print("Management IP    : {}".format(management_ip_switch))
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}command.txt'
    print(cmd_file)
    file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))

    stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

Swtitch Name        : s1
Management IP    : 63.239.135.122
s1command.txt
file_attributes: -rw-rw-r--   1 1000     1000          426 04 Sep 18:02 ?
stdout: ('Obtaining JSON from switch...\nDone\nControl utility for runtime P4 table manipulation\nRuntimeCmd: RuntimeCmd: RuntimeCmd: Setting default action of MyEgress.swtrace\naction:              add_swtrace\nruntime data:        00:00:00:01\nRuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm\nmatch key:           LPM-0a:82:04:02/32\naction:              ipv4_forward\nruntime data:        2a:5f:fb:92:07:f6\t00:01\nEntry has been added with handle 0\nRuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm\nmatch key:           LPM-0a:82:05:02/32\naction:              ipv4_forward\nruntime data:        2a:ae:c2:4c:53:d0\t00:02\nEntry has been added with handle 1\nRuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm\nmatch key:           LPM-0a:82:06:02/32\naction:              ipv4_forward\nruntime data:        

## The switches are now configured and running. Now we are going to send packets over the switches.

We are going to use `send.py` and `receive.py`. We are going to re-upload them to the servers and use them. Make sure to modify the interface names in the script accordingly.

In [22]:
h1.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h1.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')
h2.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h2.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')
h11.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h11.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')
h22.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h22.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

<SFTPAttributes: [ size=1498 uid=1000 gid=1000 mode=0o100664 atime=1662314568 mtime=1662314568 ]>

In [23]:
print(h1)

-----------------  ---------------------------------------------------------------------------------------------------------------------------------
ID                 068f63e7-91eb-4629-9ef8-cef4a11ce8e0
Name               h1
Cores              2
RAM                8
Disk               100
Image              default_ubuntu_20
Image Type         qcow2
Host               max-w5.fabric-testbed.net
Site               MAX
Management IP      63.239.135.82
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/.ssh/id_rsa -J durbek_gafurov_0000000854@bastion-1.fabric-testbed.net ubuntu@63.239.135.82
-----------------  ---------------------------------------------------------------------------------------------------------------------------------


## H1 sends packet, H2 recieves

In [41]:
print('tmux new -d \'timeout 15 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py '+h2IP.split('/')[0]+' "message100" 1\'')
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py '+h2IP.split('/')[0]+' "message100" 1\'')
print(h2.execute('sudo timeout 15 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])


tmux new -d 'timeout 15 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py 10.130.6.2 "message100" 1'
sniffing on ens7
got a packet
###[ Ethernet ]### 
  dst       = 2e:fb:22:d0:52:69
  src       = 82:31:bd:61:8f:bd
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 6
     tos       = 0x0
     len       = 42
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 63
     proto     = udp
     chksum    = 0x3b75
     src       = 10.20.5.178
     dst       = 10.130.6.2
     \options   \
      |###[ MRI ]### 
      |  copy_flag = 0
      |  optclass  = control
      |  option    = 31
      |  length    = 4
      |  count     = 0
      |  \swtraces  \
###[ UDP ]### 
        sport     = 1234
        dport     = 4321
        len       = 18
        chksum    = 0xf22b
###[ Raw ]### 
           load      = 'message100'

got a packet
###[ Ethernet ]### 
  dst       = 2e:fb:22:d0:52:69
  src       = 82:31:bd:61:8f:bd
  type      = IPv4
###[

## H1 sends packet, H11 recieves

In [37]:
h1.execute('tmux new -d \'timeout 30 watch -n 1 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py '+h11IP.split('/')[0]+' "message100" 1\'')
print(h11.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7
got a packet
###[ Ethernet ]### 
  dst       = 2a:ae:c2:4c:53:d0
  src       = 82:31:bd:61:8f:c0
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 6
     tos       = 0x0
     len       = 42
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 63
     proto     = udp
     chksum    = 0x3c75
     src       = 10.20.5.178
     dst       = 10.130.5.2
     \options   \
      |###[ MRI ]### 
      |  copy_flag = 0
      |  optclass  = control
      |  option    = 31
      |  length    = 4
      |  count     = 0
      |  \swtraces  \
###[ UDP ]### 
        sport     = 1234
        dport     = 4321
        len       = 18
        chksum    = 0xf32b
###[ Raw ]### 
           load      = 'message100'

got a packet
###[ Ethernet ]### 
  dst       = 2a:ae:c2:4c:53:d0
  src       = 82:31:bd:61:8f:c0
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 6
     tos       = 0x0
     len       = 42
     id        = 1
     flags     = 

## H2 sends packet, H1 recieves

In [26]:
h2.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py '+h1IP.split('/')[0]+' "message100" 1\'')
print(h1.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7
got a packet
###[ Ethernet ]### 
  dst       = 2a:5f:fb:92:07:f6
  src       = 82:31:bd:61:8f:c1
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 6
     tos       = 0x0
     len       = 42
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 63
     proto     = udp
     chksum    = 0x3e47
     src       = 10.20.4.224
     dst       = 10.130.4.2
     \options   \
      |###[ MRI ]### 
      |  copy_flag = 0
      |  optclass  = control
      |  option    = 31
      |  length    = 4
      |  count     = 0
      |  \swtraces  \
###[ UDP ]### 
        sport     = 1234
        dport     = 4321
        len       = 18
        chksum    = 0xf4fd
###[ Raw ]### 
           load      = 'message100'

got a packet
###[ Ethernet ]### 
  dst       = 2a:5f:fb:92:07:f6
  src       = 82:31:bd:61:8f:c1
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 6
     tos       = 0x0
     len       = 42
     id        = 1
     flags     = 

In [30]:
h2.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py '+h1IP.split('/')[0]+' "message100" 1\'')
print(h1.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7
got a packet
###[ Ethernet ]### 
  dst       = 2a:5f:fb:92:07:f6
  src       = 82:31:bd:61:8f:c1
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 6
     tos       = 0x0
     len       = 42
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 63
     proto     = udp
     chksum    = 0x3e47
     src       = 10.20.4.224
     dst       = 10.130.4.2
     \options   \
      |###[ MRI ]### 
      |  copy_flag = 0
      |  optclass  = control
      |  option    = 31
      |  length    = 4
      |  count     = 0
      |  \swtraces  \
###[ UDP ]### 
        sport     = 1234
        dport     = 4321
        len       = 18
        chksum    = 0xf4fd
###[ Raw ]### 
           load      = 'message100'

got a packet
###[ Ethernet ]### 
  dst       = 2a:5f:fb:92:07:f6
  src       = 82:31:bd:61:8f:c1
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 6
     tos       = 0x0
     len       = 42
     id        = 1
     flags     = 

## H22 sends packet, H11 recieves

In [35]:

h22.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py '+h11IP.split('/')[0]+' "message100" 1\'')
print(h11.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7
got a packet
###[ Ethernet ]### 
  dst       = 2a:ae:c2:4c:53:d0
  src       = 82:31:bd:61:8f:c0
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 6
     tos       = 0x0
     len       = 42
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 63
     proto     = udp
     chksum    = 0x3c77
     src       = 10.20.5.176
     dst       = 10.130.5.2
     \options   \
      |###[ MRI ]### 
      |  copy_flag = 0
      |  optclass  = control
      |  option    = 31
      |  length    = 4
      |  count     = 0
      |  \swtraces  \
###[ UDP ]### 
        sport     = 1234
        dport     = 4321
        len       = 18
        chksum    = 0xf32d
###[ Raw ]### 
           load      = 'message100'

got a packet
###[ Ethernet ]### 
  dst       = 2a:ae:c2:4c:53:d0
  src       = 82:31:bd:61:8f:c0
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 6
     tos       = 0x0
     len       = 42
     id        = 1
     flags     = 

## Delete Slice

In [29]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Fail: {e}")
#     traceback.print_exc()